# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 0.Setting Colab Method for future model developing
Firstly, run the following block to mount the drive to the colab. Then, drag the data folder/**eval.py** to the "Colab Folder Space" to ensure the code runs successfully.

If data folder updated, attempt to forcibly remount, call `drive.mount("/content/drive", force_remount=True)`.


In [28]:
# from google.colab import drive
# drive.mount('/content/drive')

# 1.DataSet Processing

## 1.1 Reading and gathering data

Using `json` package reading and gathering claims and evidences, then print an output.

In [1]:
import json
from collections import Counter
from statistics import mean

with open('data/train-claims.json', 'r') as input_file:
    train_claim_data = json.load(input_file)

# Read in development data (claim)
with open('data/dev-claims.json', 'r') as input_file:
    dev_claim_data = json.load(input_file)

# Read in test data (claim)
with open('data/test-claims-unlabelled.json', 'r') as input_file:
    test_claim_data = json.load(input_file)

# Read in evidence data
with open('data/evidence.json', 'r') as input_file:
    evi_data = json.load(input_file)

#EDA


claim_count = 0
evi_count = 0
claim_length = []
evidence_count = []
evidence_length = []
labels = []

for key,value in train_claim_data.items():
    claim_count+=1
    claim_length.append(len(value["claim_text"]))
    evidence_count.append(len(value["evidences"]))
    evidence_length += [len(evi_data[x]) for x in value["evidences"]]
    labels.append(value["claim_label"])

for key,value in evi_data.items():
    evi_count+=1

print("claim count: ",claim_count)
print("evidence count: ",evi_count)
print("max claim length: ",max(claim_length))
print("min claim length: ",min(claim_length))
print("mean claim length: ",mean(claim_length))
print("max evidence count: ",max(evidence_count))
print("min evidence count: ",min(evidence_count))
print("mean evidence count: ",mean(evidence_count))
print("max evidence length: ",max(evidence_length))
print("min evidence length: ",min(evidence_length))
print("mean evidence length: ",mean(evidence_length))
print(Counter(labels))



inside = 0
outside = 0

train_evi_id = []
for claim_id,claim_value in train_claim_data.items():
    train_evi_id=train_evi_id+claim_value['evidences']

for claim_id,claim_value in dev_claim_data.items():
    test_evi_id=claim_value['evidences']
    for e in test_evi_id:
        if e in train_evi_id:
            inside += 1
        else:
            outside += 1
print("Dev evi inside train evi", inside)
print("Dev evi outside train evi", outside)

full_evidence_id = list(evi_data.keys())
full_evidence_text  = list(evi_data.values())
train_claim_id = list(train_claim_data.keys())
train_claim_text  = [ v["claim_text"] for v in train_claim_data.values()]
print("Train claim count: ",len(train_claim_id))


claim count:  1228
evidence count:  1208827
max claim length:  332
min claim length:  26
mean claim length:  122.95521172638436
max evidence count:  5
min evidence count:  1
mean evidence count:  3.3566775244299674
max evidence length:  1979
min evidence length:  13
mean evidence length:  173.5
Counter({'SUPPORTS': 519, 'NOT_ENOUGH_INFO': 386, 'REFUTES': 199, 'DISPUTED': 124})
Dev evi inside train evi 163
Dev evi outside train evi 328
Train claim count:  1228


In [3]:
print(evi_data["evidence-67732"])

[citation needed] South Australia has the highest retail price for electricity in the country.


## 1.2 Data preprocessing

### Implementing preprocessing fuctions

In [2]:
import nltk
import string
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    return lemma if lemma != word else lemmatizer.lemmatize(word, 'n')

def is_pure_english(text):
    english_letters = set(string.ascii_letters)
    cleaned_text = ''.join(char for char in text if char.isalpha() or char.isspace())
    return all(char in english_letters or char.isspace() for char in cleaned_text)

def remove_non_eng(dictionary):
    eng_data = {}
    for key, value in dictionary.items():
        if is_pure_english(value):
            eng_data[key] = value
    return eng_data

def contains_climate_keywords(text, keywords):
    text = text.lower()
    for keyword in keywords:
        if re.search(r"\b" + re.escape(keyword) + r"\b", text):
            return True
    return False

def filter_climate_related(dictionary, keywords):
    cs_data = {}
    for key, value in dictionary.items():
        if contains_climate_keywords(value, keywords):
            cs_data[key] = value
    return cs_data

def text_preprocessing(text, remove_stopwords=False):
    words = [lemmatize(w) for w in text.lower().split()]
    if remove_stopwords:
        words = [w for w in words if w not in stopwords]
    return " ".join(words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ABC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ABC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Implementing **Claim data preprocessing** and **Evidence data preprocessing** functions

In [3]:
# 权威网站 https://www.ucdavis.edu/climate/definitions
climate_keywords = [
    "climate", "environment", "global warming", "greenhouse effect", "carbon", "co2", "carbon dioxide",
    "methane", "renewable energy", "sustainability", "ecology", "biodiversity", "fossil fuels",
    "emissions", "air quality", "ozone", "solar energy", "wind energy", "climate change", "climate crisis",
    "climate adaptation", "climate mitigation", "ocean", "sea levels", "ice melting", "deforestation",
    "reforestation", "pollution"," electricity","energy","solar","wind","renewable","fossil","fuel","emission","air","quality","ozone","solar","wind","climate","change","crisis","adaptation","mitigation","ocean","sea","level","ice","melt","deforestation",
]

# def filter_evidence_by_train(train_claim_data, evidence_data):

#     # Collect all evidence ids in the training set
#     train_evidence_ids = set()

#     for claim in train_claim_data.values():
#         train_evidence_ids.update(claim['evidences'])

#     # filter evidence data by the evidence ids in the training set
#     filtered_evidence_data = {key: value for key, value in evidence_data.items() if key in train_evidence_ids}

#     return filtered_evidence_data

def preprocess_claim_data(claim_data, existed_evidences_id=None):
    claim_data = remove_non_eng(claim_data)
    claim_data_text = []
    claim_data_id = []
    claim_data_label = []
    claim_evidences = []

    for key in claim_data.keys():
        claim_data[key]["claim_text"] = text_preprocessing(claim_data[key]["claim_text"])
        claim_data_text.append(claim_data[key]["claim_text"])
        claim_data_id.append(key)

        if "claim_label" in claim_data[key]:
            claim_data_label.append(claim_data[key]["claim_label"])
        else:
            claim_data_label.append(None)

        if existed_evidences_id and "evidences" in claim_data[key]:
            valid_evidences = [existed_evidences_id[i] for i in claim_data[key]["evidences"] if i in existed_evidences_id]
            claim_evidences.append(valid_evidences)
        else:
            claim_evidences.append([])
    
    return claim_data_text, claim_data_id, claim_data_label, claim_evidences

# def preprocess_evi_data(evi_data, climate_keywords, train_claim_data):
#     evi_data = remove_non_eng(evi_data)
#     # cs_evi_data = filter_climate_related(evi_data, climate_keywords)

#     # filter evidence data by the evidence ids in the training set
#     # train_evi_data = filter_evidence_by_train(train_claim_data, cs_evi_data)

#     for key in evi_data.keys():
#         evi_data[key] = text_preprocessing(evi_data[key], remove_stopwords=True)

#     cleaned_evidence_text = list(evi_data.values())
#     cleaned_evidence_id = list(evi_data.keys())

#     return cleaned_evidence_text, cleaned_evidence_id

def preprocess_evi_data(evi_data, climate_keywords):
    evi_data = remove_non_eng(evi_data)
    cs_evi_data = filter_climate_related(evi_data, climate_keywords)

    # filter evidence data by the evidence ids in the training set
    # train_evi_data = filter_evidence_by_train(train_claim_data, cs_evi_data)

    for key in cs_evi_data.keys():
        cs_evi_data[key] = text_preprocessing(cs_evi_data[key], remove_stopwords=True)

    cleaned_evidence_text = list(cs_evi_data.values())
    cleaned_evidence_id = list(cs_evi_data.keys())

    return cleaned_evidence_text, cleaned_evidence_id

### Start dataset preprocessing

In [4]:
# Preprocessing the evidence data, split the data into text and id
# cleaned_evidence_text, cleaned_evidence_id = preprocess_evi_data(evi_data, climate_keywords, train_claim_data)

cleaned_evidence_text, cleaned_evidence_id = preprocess_evi_data(evi_data, climate_keywords)

# Create a dictionary to map evidence ID to index
evidences_id_dict = {evidence_id: index for index, evidence_id in enumerate(cleaned_evidence_id)}

# Preprocessing the claim data, split the data into text, id, label and evidences
train_claim_text, train_claim_id, train_claim_label, train_claim_evidences = preprocess_claim_data(train_claim_data, evidences_id_dict)

dev_claim_text, dev_claim_id, dev_claim_label, dev_claim_evidences = preprocess_claim_data(dev_claim_data, evidences_id_dict)

test_claim_text, test_claim_id, _, _ = preprocess_claim_data(test_claim_data)


In [5]:
for i in dev_claim_evidences:
    print(i)
print(len(dev_claim_evidences))

[3065]
[44825, 48597, 9350, 31415, 37401]
[]
[]
[2418, 11925, 42548, 19070]
[41043, 41128]
[50766, 20539, 28072, 26443, 33310]
[38516, 51685]
[29618]
[38012, 4842, 14562]
[9430]
[10944]
[4014, 50675, 50107]
[42218, 44297, 393, 29354, 35467]
[7116, 47166]
[14743, 48829, 31497]
[31565]
[44694, 45292, 16548]
[32836, 1820, 49762, 42881, 43002]
[53878, 22924, 47558, 4583, 53772]
[35201, 18514]
[10100, 50063]
[33569, 33857, 1008]
[16548, 19050]
[6954, 22601, 15144]
[14226]
[29504, 24856, 49068]
[3460]
[48697, 26135]
[20224, 29523, 43241, 35995]
[24743]
[]
[]
[170, 46679, 28088]
[]
[]
[42489, 47180]
[38499, 35819]
[]
[7801, 21457]
[52370]
[5453]
[30613, 33440, 15894, 7786]
[1214, 31934]
[9994]
[53772]
[41128, 33370, 7642]
[47654, 53343, 42684]
[]
[]
[20539]
[]
[20659]
[54123, 27844, 986]
[24856, 393]
[45771, 1382]
[36815, 45922, 27280, 24287]
[]
[]
[47166]
[53970, 21739]
[53343, 39598, 15248]
[47058, 25758]
[1572]
[32728, 52558]
[35435, 11371]
[42489, 50779, 20539, 49068]
[41973, 47282]
[2713

In [6]:
print("Train: ", train_claim_evidences)
print("Dev: ", dev_claim_evidences)

Train:  [[19904, 53631, 573], [15248, 50675], [23802], [52869, 35127, 24221, 15888, 45304], [], [10153, 47158, 16119, 22133, 27407], [], [1332], [], [19754], [9796], [9991, 40656, 26941, 15231], [17407, 48486, 43840], [23176, 45804, 35535, 45368, 39487], [22313], [], [7915, 23176, 45368], [16548, 51591, 20591, 4119, 36575], [24918], [53970], [8295], [39598], [5693, 41152, 13912], [37913, 49862, 12818], [4014, 53215, 27738], [], [45499], [26994, 50150], [34378], [], [14939], [], [42687, 32620], [45192, 3136, 28289, 53859, 51682], [], [17044, 5390, 40237, 35034, 39051], [], [35467], [], [5657, 34324, 43723], [50684, 29702, 48077, 46600], [16548, 2974, 21633], [23007, 46114, 2455], [30406, 31444], [49867, 41577, 24183, 33370], [52546, 21739, 53010], [39711], [28684, 12952, 16454], [13473], [28256, 17749], [9879, 34594], [18140, 19193], [27227], [37911, 32121, 9873, 21633], [], [17333, 6416, 27829], [6008, 21477], [44851], [18774], [50152], [37547, 50675, 17213, 32464], [3090], [37058], [4

In [7]:
print("Train claim count: ",len(train_claim_text))
print("Dev claim count: ",len(dev_claim_text))
print("Test claim count: ",len(test_claim_text))
print("Evidence count: ",len(cleaned_evidence_text))

Train claim count:  1228
Dev claim count:  154
Test claim count:  153
Evidence count:  54272


## 1.3 Development Set Prediction

In this section, we perform the main tasks of the project on the development set:

1. **Evidence Retrieval**: For each claim, find the most relevant evidence from the corpus.
2. **Claim Classification**: Predict the label for each claim based on the retrieved evidence and the claim's similarity to the training claims.

The code uses TF-IDF vectorization and cosine similarity to measure the relevance between claims and evidence, and between development and training claims. The most similar evidence and training claims are used for prediction.

In [16]:
import operator
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import euclidean

# Creating two vectorizer
evidence_tfidf_vectorizer = TfidfVectorizer(max_features=54000, use_idf=True)

# fit the vectorizer on the evidence data
evidence_tfidf_vectorizer.fit(cleaned_evidence_text)

# Transform cleaned_evidence_text
transformed_evidence = evidence_tfidf_vectorizer.transform(cleaned_evidence_text)

# Transform claim data
train_claim_tfidf = evidence_tfidf_vectorizer.transform(train_claim_text)
dev_claim_tfidf = evidence_tfidf_vectorizer.transform(dev_claim_text)
test_claim_tfidf = evidence_tfidf_vectorizer.transform(test_claim_text)

In [17]:
print("Transformed evidence shape: ", transformed_evidence.shape)
print("Transformed train claim shape: ", train_claim_tfidf.shape)
print("Transformed dev claim shape: ", dev_claim_tfidf.shape)

Transformed evidence shape:  (54272, 54000)
Transformed train claim shape:  (1228, 54000)
Transformed dev claim shape:  (154, 54000)


In [18]:
# Calculate cosine similarity between train claims and evidence
train_similarity = cosine_similarity(train_claim_tfidf, transformed_evidence)

# Calculate cosine similarity between dev claims and evidence
dev_similarity = cosine_similarity(dev_claim_tfidf, transformed_evidence)

# Calculate cosine similarity between test claims and evidence
test_similarity = cosine_similarity(test_claim_tfidf, transformed_evidence)


In [19]:
print("Train similarity:", train_similarity)
print("Test similarity: ", test_similarity)

Train similarity: [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00960401 0.         0.         ... 0.         0.03327222 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
Test similarity:  [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.05392114 ... 0.         0.         0.        ]
 [0.01999309 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.07248254 ... 0.14812693 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [41]:
# # Calculate Euclidean distance between train claims and evidence
# train_distance = np.zeros((train_claim_tfidf.shape[0], transformed_evidence.shape[0]))
# for i in range(train_claim_tfidf.shape[0]):
#     for j in range(transformed_evidence.shape[0]):
#         train_distance[i, j] = euclidean(train_claim_tfidf[i].toarray().ravel(), transformed_evidence[j].toarray().ravel())

# # Calculate Euclidean distance between dev claims and evidence
# dev_distance = np.zeros((dev_claim_tfidf.shape[0], transformed_evidence.shape[0]))
# for i in range(dev_claim_tfidf.shape[0]):
#     for j in range(transformed_evidence.shape[0]):
#         dev_distance[i, j] = euclidean(dev_claim_tfidf[i].toarray().ravel(), transformed_evidence[j].toarray().ravel())

# # Calculate Euclidean distance between test claims and evidence
# test_distance = np.zeros((test_claim_tfidf.shape[0], transformed_evidence.shape[0]))
# for i in range(test_claim_tfidf.shape[0]):
#     for j in range(transformed_evidence.shape[0]):
#         test_distance[i, j] = euclidean(test_claim_tfidf[i].toarray().ravel(), transformed_evidence[j].toarray().ravel())

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [20]:
def spliting_dataset(similarity, claim_texts, claim_evidences, evidence_texts, evidence_ids, top_k=5, neg_ratio=1, train_mode=False):
    dataset = []
    labels = []
    evi_label = []
    for i in range(similarity.shape[0]):
        claim_text = claim_texts[i]
        
        top_evidences = np.argsort(-similarity[i])[:top_k]
        
        for evidence_index in top_evidences:
            evidence_text = evidence_texts[evidence_index]
            evidence_id = evidence_ids[evidence_index]
            evi_label.append(evidence_id)
            dataset.append("[cls]" + claim_text + "[sep]" + evidence_text)
            
            if claim_evidences is not None and evidence_index in claim_evidences[i]:
                labels.append(1)
            else:
                labels.append(0)
        
        if train_mode and claim_evidences is not None:
            for evidence_index in claim_evidences[i]:
                if evidence_index not in top_evidences:
                    evidence_text = evidence_texts[evidence_index]
                    dataset.append("[cls]" + claim_text + "[sep]" + evidence_text)
                    labels.append(1)
        
        neg_samples_num = int(neg_ratio * top_k)
        neg_evidences = np.random.choice(
            [j for j in range(similarity.shape[1]) if j not in top_evidences and (not train_mode or j not in claim_evidences[i])], 
            neg_samples_num
        )
        
        for evidence_index in neg_evidences:
            evidence_text = evidence_texts[evidence_index]
            dataset.append("[cls]" + claim_text + "[sep]" + evidence_text)
            labels.append(0)
    
    return dataset, labels,evi_label

In [21]:
train_dataset, train_dataset_labels ,train_dataset_eviid= spliting_dataset(
    train_similarity, train_claim_text, train_claim_evidences, cleaned_evidence_text, cleaned_evidence_id,
    top_k=50, neg_ratio=1, train_mode=True
)
dev_dataset, dev_dataset_labels, dev_dataset_eviid = spliting_dataset(
    dev_similarity, dev_claim_text, dev_claim_evidences, cleaned_evidence_text, cleaned_evidence_id,
    top_k=50, neg_ratio=1, train_mode=False
)
test_dataset, test_dataset_labels, test_dataset_eviid = spliting_dataset(
    test_similarity, test_claim_text, None, cleaned_evidence_text, cleaned_evidence_id,
    top_k=50, neg_ratio=1, train_mode=False
)

In [22]:
for i in dev_dataset_eviid:
    if i == "evidence-67732":
        print(i)

evidence-67732


In [23]:
print(dev_dataset_eviid[0])
print(len(dev_dataset_eviid)/50)
print(dev_dataset[0])
print(dev_dataset[101])
print(len(dev_claim_text))

evidence-839635
154.0
[cls][south australia] have the most expensive electricity in the world.[sep]couple consumption disparages, lead mix "have" "have not" electric energy states.
[cls]when 3 per cent of total annual global emission of carbon dioxide be from human and australia prod­uces 1.3 per cent of this 3 per cent, then no amount of emission reductio­n here will have any effect on global climate.[sep]example, "australian national greenhouse gas inventory" (nggi) prepare compliance requirement indicate energy sector account 69 per cent australia’s emissions, agriculture 16 per cent lulucf six per cent.
154


In [46]:
# def spliting_dataset(similarity, claim_texts, claim_evidences, evidence_texts, threshold=0.5, neg_ratio=1):
#     dataset = []
#     labels = []
    
#     # Calculate the mean similarity for each claim
#     mean_similarities = np.mean(similarity, axis=1)
    
#     # Based on the similarity matrix, find the evidence that has similarity higher than the threshold
#     for i in range(similarity.shape[0]):
#         claim_text = claim_texts[i]
        
#         # Find the evidence with similarity higher than the threshold
#         top_evidences = np.where(similarity[i] > threshold * mean_similarities[i])[0]
        
#         # Add the top evidence to the dataset, label as 1
#         for evidence_index in top_evidences:
#             evidence_text = evidence_texts[evidence_index]
#             dataset.append("[cls]" + claim_text + "[sep]" + evidence_text)
#             labels.append(1)
        
#         # If the claim has evidences, add the evidence to the dataset, label as 1
#         if claim_evidences is not None:
#             for evidence_index in claim_evidences[i]:
#                 evidence_text = evidence_texts[evidence_index]
#                 dataset.append("[cls]" + claim_text + "[sep]" + evidence_text)
#                 labels.append(1)
        
#         # Randomly sample negative samples, label as 0
#         neg_samples_num = int(neg_ratio * len(top_evidences))
        
#         # Randomly sample negative samples that are not in the top evidence
#         neg_evidences = np.random.choice(
#             [j for j in range(similarity.shape[1]) if j not in top_evidences], 
#             neg_samples_num
#         )
        
#         # Add the negative samples to the dataset
#         for evidence_index in neg_evidences:
#             evidence_text = evidence_texts[evidence_index]
#             dataset.append("[cls]" + claim_text + "[sep]" + evidence_text)
#             labels.append(0)
    
#     return dataset, labels

In [47]:
# train_dataset, train_dataset_labels = spliting_dataset(
#     train_similarity, train_claim_text, train_claim_evidences, cleaned_evidence_text, threshold=0.8, neg_ratio=1.1
# )
# dev_dataset, dev_dataset_labels = spliting_dataset(
#     dev_similarity, dev_claim_text, dev_claim_evidences, cleaned_evidence_text, threshold=0.8, neg_ratio=1.1
# )
# test_dataset, test_dataset_labels = spliting_dataset(
#     test_similarity, test_claim_text, None, cleaned_evidence_text, threshold=0.8, neg_ratio=1.1
# )

In [24]:
# Convert the dataset labels to numpy array
train_label_array = np.array(train_dataset_labels)
dev_label_array = np.array(dev_dataset_labels)
test_label_array = np.array(test_dataset_labels)

In [25]:
from collections import Counter
print(Counter(train_label_array))
print(Counter(dev_label_array))

Counter({0: 122019, 1: 2533})
Counter({0: 15300, 1: 100})


In [26]:
122033 / 2533

48.17726016581129

In [27]:
15302 / 98

156.14285714285714

In [28]:
# need to install
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(train_dataset)

In [29]:
vocab_size = len(tokenizer.word_index) + 1 # 0 is padding token
print("Vocab size: ", vocab_size)

Vocab size:  48940


In [30]:
# Convert the text data to sequence
train_sequence = tokenizer.texts_to_sequences(train_dataset)
dev_sequence = tokenizer.texts_to_sequences(dev_dataset)
test_sequence = tokenizer.texts_to_sequences(test_dataset)

In [31]:
longest_train_sequence = 0
for i in train_sequence:
    longest_train_sequence = max(longest_train_sequence, len(i))

longest_dev_sequence = 0
for i in dev_sequence:
    longest_dev_sequence = max(longest_dev_sequence, len(i))


In [32]:
print("Longest train sequence: ", longest_train_sequence)
print("Longest dev sequence: ", longest_dev_sequence)

Longest train sequence:  238
Longest dev sequence:  213


In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padding_length = min(longest_train_sequence, longest_dev_sequence) + 1

padded_train_sequence = pad_sequences(train_sequence, maxlen=padding_length, padding='post')
padded_dev_sequence = pad_sequences(dev_sequence, maxlen=padding_length, padding='post')
padded_test_sequence = pad_sequences(test_sequence, maxlen=padding_length, padding='post')

In [52]:
print("Padded train sequence shape: ", padding_length)

Padded train sequence shape:  214


In [55]:
# from workshop
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Attention
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2

# def focal_loss(gamma=2., alpha=4.):
#     def focal_loss_fixed(y_true, y_pred):
#         pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
#         pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
#         return -tf.math.reduce_sum(alpha * tf.pow(1. - pt_1, gamma) * tf.math.log(pt_1))-tf.math.reduce_sum((1-alpha) * tf.pow( pt_0, gamma) * tf.math.log(1. - pt_0))
#     return focal_loss_fixed

embedding_dim = 100
hidden_dim = 200

#model definition
# feedforward network (MLP)
model = Sequential(name="retrieval_cls_lstm")

model.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim,
                           input_length=padding_length, embeddings_regularizer=l2(0.02)))

model.add(layers.Bidirectional(
    layers.RNN(
        layers.LSTMCell(hidden_dim, dropout=0.4, kernel_regularizer=l2(0.02)),
        return_sequences=True
    )
))

model.add(layers.GlobalMaxPooling1D())

# model.add(layers.Dropout(0.4))

# model.add(layers.Bidirectional(LSTM(hidden_dim, return_sequences=True, dropout=0.4, kernel_regularizer=l2(0.02))))

# model.add(layers.Bidirectional(LSTM(hidden_dim, dropout=0.4)))
# model.add(layers.GlobalMaxPooling1D())

model.add(layers.Dropout(0.4))
model.add(layers.Dense(hidden_dim, activation='tanh', kernel_regularizer=l2(0.02), bias_regularizer=l2(0.02)))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(1, activation='sigmoid'))

#since it's a binary classification problem, we use a binary cross entropy loss here
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras.metrics.Recall()])
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='binary_crossentropy', optimizer='adam')

decay_steps = 2000
learning_rate = 1e-4
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    learning_rate, decay_steps
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
# model.compile(loss=focal_loss(gamma=2., alpha=0.25), optimizer=optimizer)
model.compile(loss='binary_crossentropy', optimizer=optimizer)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='min')
model.summary()

Model: "retrieval_cls_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 214, 100)          4894000   
                                                                 
 bidirectional_13 (Bidirecti  (None, 214, 400)         481600    
 onal)                                                           
                                                                 
 global_max_pooling1d_3 (Glo  (None, 400)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_5 (Dropout)         (None, 400)               0         
                                                                 
 dense_8 (Dense)             (None, 200)               80200     
                                                                 
 dropout_6 (Dropout)         (None, 200)        

In [56]:
# Train the model

model.fit(padded_train_sequence,train_label_array,epochs=10,validation_data=(padded_dev_sequence, dev_label_array),verbose=True,batch_size=600,callbacks=[earlystopping])

Epoch 1/10
208/208 [==============================] - 158s 747ms/step - loss: 54.8238 - val_loss: 27.6425
Epoch 2/10
208/208 [==============================] - 154s 740ms/step - loss: 15.9438 - val_loss: 8.0340
Epoch 3/10
208/208 [==============================] - 156s 749ms/step - loss: 4.8315 - val_loss: 2.6134
Epoch 4/10
208/208 [==============================] - 154s 739ms/step - loss: 1.7545 - val_loss: 1.0722
Epoch 5/10
208/208 [==============================] - 154s 741ms/step - loss: 0.8324 - val_loss: 0.5644
Epoch 6/10
208/208 [==============================] - 156s 747ms/step - loss: 0.5087 - val_loss: 0.3676
Epoch 7/10
208/208 [==============================] - 154s 741ms/step - loss: 0.3765 - val_loss: 0.2851
Epoch 8/10
208/208 [==============================] - 156s 751ms/step - loss: 0.3198 - val_loss: 0.2504
Epoch 9/10
208/208 [==============================] - 154s 742ms/step - loss: 0.2990 - val_loss: 0.2398
Epoch 10/10
208/208 [==============================] - 155s 7

In [61]:
# Save the model
# model.save('retrieval_cls_lstm')

# Load the model
# model = tf.keras.models.load_model('retrieval_cls_lstm')

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [57]:
# Start prediction

dev_predictions = model.predict(padded_dev_sequence, batch_size=64)
test_predictions = model.predict(padded_test_sequence, batch_size=64)


240/240 [==============================] - 18s 73ms/step


In [58]:
print(dev_predictions[:20])
print(test_predictions[:5])
print(len(dev_predictions))
print(len(test_predictions))

[[0.01986083]
 [0.01986079]
 [0.01986112]
 [0.01986082]
 [0.01986025]
 [0.01986099]
 [0.01986079]
 [0.01986031]
 [0.01986039]
 [0.01986043]
 [0.01986113]
 [0.01986109]
 [0.01986041]
 [0.01986043]
 [0.01986025]
 [0.01986126]
 [0.01986117]
 [0.01986117]
 [0.01986112]
 [0.01986024]]
[[0.01986039]
 [0.01986079]
 [0.0198609 ]
 [0.01986035]
 [0.0198607 ]]
15400
15300


In [59]:
print(len(dev_dataset)/100)
print(len(dev_claim_text))
print(dev_dataset[0])
print(dev_claim_text[0])
print(dev_claim_text[2])
print(dev_dataset_eviid[0])

154.0
154
[cls][south australia] have the most expensive electricity in the world.[sep]couple consumption disparages, lead mix "have" "have not" electric energy states.
[south australia] have the most expensive electricity in the world.
this mean that the world be now 1c warmer than it be in pre-industrial time
evidence-839635


In [60]:
def evidence_predicts(predictions, evi_id, select_k):
    start_idx = 0
    pred_evidences = []
    count_idx = 0
    while start_idx < len(predictions):
        end_idx = start_idx + 100
        cur_logits = predictions[start_idx: end_idx]
        cur_top_ids = np.argpartition(-cur_logits.squeeze(), select_k)[:select_k].tolist()
        for i in cur_top_ids:
            pred_evidences.append(evi_id[i])
        start_idx = end_idx
        count_idx += 1
    return pred_evidences

In [61]:
pred_dev_claims = {}
pred_test_claims = {}

all_pred_eviid = evidence_predicts(dev_predictions, dev_dataset_eviid, 6)

cluster_index = 0
for claim_t in dev_claim_text:
    pred_dev_claims[claim_t] = all_pred_eviid[cluster_index:cluster_index+6]
    cluster_index += 6
    
print(next(iter(pred_dev_claims.keys())))

prdict_list = list(pred_dev_claims.values())
print(prdict_list)

[south australia] have the most expensive electricity in the world.
[['evidence-851919', 'evidence-379089', 'evidence-396010', 'evidence-380007', 'evidence-793193', 'evidence-979811'], ['evidence-1075957', 'evidence-334038', 'evidence-941912', 'evidence-323649', 'evidence-998626', 'evidence-989072'], ['evidence-551116', 'evidence-291834', 'evidence-167278', 'evidence-839635', 'evidence-223713', 'evidence-660083'], ['evidence-157374', 'evidence-741504', 'evidence-839635', 'evidence-31767', 'evidence-1075957', 'evidence-205762'], ['evidence-731175', 'evidence-730396', 'evidence-941912', 'evidence-291834', 'evidence-996773', 'evidence-832424'], ['evidence-839635', 'evidence-329731', 'evidence-693286', 'evidence-204182', 'evidence-962087', 'evidence-407584'], ['evidence-432019', 'evidence-241551', 'evidence-851919', 'evidence-731175', 'evidence-680459', 'evidence-1070127'], ['evidence-505092', 'evidence-1028051', 'evidence-452156', 'evidence-1034764', 'evidence-293046', 'evidence-839635'],

In [62]:
# # Update the dev JSON file
with open('data/dev_predict.json', 'r') as f:
    dev_claims = json.load(f)
predict_id = 0

for claim_id, claim_info in dev_claims.items():
    claim_info["evidences"] = prdict_list[predict_id]
    predict_id += 1
    
print(next(iter(dev_claims.items())))
with open('data/dev_predict.json', 'w') as f:
    json.dump(dev_claims, f)
# for claim_id, evidence_indices in zip(dev_claim_id, dev_top_evidence_indices):
#     top_evidence_ids = [cleaned_evidence_id[idx] for idx in evidence_indices]
#     dev_claims[claim_id]['evidences'] = top_evidence_ids

# with open('data/dev_predict.json', 'w') as f:
#     json.dump(dev_claims, f)


('claim-752', {'claim_text': '[South Australia] has the most expensive electricity in the world.', 'claim_label': 'NOT_ENOUGH_INFO', 'evidences': ['evidence-851919', 'evidence-379089', 'evidence-396010', 'evidence-380007', 'evidence-793193', 'evidence-979811']})


In [68]:
# # Update the test JSON file
# with open('data/test-claims-unlabelled.json', 'r') as f:
#     test_claims = json.load(f)

# for claim_id, evidence_indices in zip(test_claim_id, test_top_evidence_indices):
#     top_evidence_ids = [cleaned_evidence_id[idx] for idx in evidence_indices]
#     test_claims[claim_id]['evidences'] = top_evidence_ids

# with open('data/test-claims-unlabelled.json', 'w') as f:
#     json.dump(test_claims, f)

In [63]:
# %%cmd
# python eval.py --predictions dev-claims-baseline.json --groundtruth dev-claims.json
# python eval.py --predictions dev_predict.json --groundtruth dev-claims.json


import subprocess

# proc = subprocess.Popen(["python", "eval.py", "--predictions", "data\dev_predict.json", "--groundtruth", "data\dev-claims.json"
# ], stdout=subprocess.PIPE, shell=True)
# (out, err) = proc.communicate()
# print(str(out))

# 高自动化模型/预处理选择，可以自动读取准确度
output = subprocess.check_output("python eval.py --predictions data/dev_predict.json --groundtruth data/dev-claims.json", shell=True)
output_str = output.decode('utf-8')

# Split the output into lines
output_lines = output_str.strip().split('\n')

# Format the output
formatted_lines = []
for line in output_lines:
    metric, value = line.split('=')
    metric = metric.strip()
    value = value.strip()
    formatted_line = f"{metric}: {value}"
    formatted_lines.append(formatted_line)

# Join the formatted lines into a single string
formatted_output = '\n'.join(formatted_lines)
print(formatted_output)

Evidence Retrieval F-score (F): 0.0
Claim Classification Accuracy (A): 0.474025974025974
Harmonic Mean of F and A: 0.0


## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*